# Dnude example

# Setup 

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import subprocess
from datetime import date
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)

In [ ]:
# analysis_version = "2017_06_22_more_mapped"
# analysis_version = "2017_06_26"
# analysis_version = "2017_06_29"
analysis_version = "prepared_for_conversation"
project_dir_absolute = Path('/data/Dnude/francois') # needs to be pathlib.Path object
# project_dir_absolute = Path('~/data/francois') # needs to be pathlib.Path object
project_dir_absolute

In [ ]:
# Generic project structure
project_dir = Path(project_dir_absolute.name)
dicoms_dir = Path('anal/dicoms') # place dicoms at this location
scripts_dir  = Path('anal/heudiconv_files')
heuristics_script = scripts_dir.joinpath('heuristics/fmrif_heuristics_' + analysis_version + '.py')
default_heuristics = heuristics_script.with_name('convertall.py')

sing_image = scripts_dir.joinpath('singularity_images/nipy_heudiconv-2017-05-27-2471285b9681.img')

dicom_extension = '.tar'
# path inside container
outdir = Path("/data/bids_" + analysis_version) 
# on local filesystem
output_of_heudiconv = Path(outdir.name) 

outdir_gen = Path(outdir.as_posix() + '_generic')
output_of_heudiconv_gen = Path(outdir_gen.name)

In [ ]:
%pwd
%cd {project_dir_absolute.as_posix()}
%pwd

## Conda environment used

The conda environment used for this analysis can be recreated using the above yml file and the command :

# Generating mapping 

In [ ]:
def add_bids_ses(df):
    df = df.assign(bids_ses = ['{num:02d}'.format(num = 1 + i) for i in range(len(df))])
    return df

df_dicoms = pd.DataFrame({'dicom_path' : [p.as_posix() for p in dicoms_dir.glob('*' + dicom_extension)]})

# the layout of the filename matters 
# for the construction of the singularity command later on.
df_dicoms = pd.concat(
    [df_dicoms,
     df_dicoms.dicom_path.
     str.extract(
         '.*-(?P<patient_id>[0-9]{,8})-(?P<date>[0-9]{8})-(?P<exam_id>[A-Z0-9]*)-.*' + dicom_extension,
         expand=True)],
    axis = 1)


df_subs = df_dicoms.drop_duplicates('patient_id')[['patient_id']]
df_subs = df_subs.assign(bids_subj = ['{x:03d}'.format(x=x) for x in range(1, 1 + len(df_subs))])
df_subs
df_dicoms = pd.merge(df_dicoms,df_subs,on='patient_id')
df_bids = (
    df_dicoms[['dicom_path','bids_subj','date','exam_id','patient_id']].
    assign(exam_id_as_int = lambda df: df.exam_id.astype(int)).
    sort_values(['bids_subj','exam_id_as_int']).
    groupby(
         ['bids_subj'],
         as_index = False).
     apply(add_bids_ses)
)
df_bids['dicom_template'] = \
'/data/' + dicoms_dir.as_posix() + \
'/*-' + df_bids.patient_id + '-' + \
df_bids.date + '-' + df_bids.exam_id + \
'-DICOM' + dicom_extension
df_bids.dicom_template

df_bids.to_csv(dicoms_dir.joinpath('bids_mapping' +analysis_version + '.tsv'),sep='\t')

df_bids

In [ ]:
len(df_bids)

# Running heudiconv

## Function for generating singularity commands

In [ ]:
def generate_singularity_command(df_row,project_dir=None,heuristics_script=None,output_dir=None,conv_dir=None,\
                                 anon_script=None,conversion=False,minmeta=False, sing_image=None):
    heuristics_script = Path('/data').joinpath(heuristics_script).as_posix()
    project_dir = Path(project_dir).as_posix()
    output_dir = Path(output_dir).as_posix()
    cmd = 'module load singularity;' + \
    'singularity exec' + \
    ' --bind ' + project_dir + ':/data' + \
    ' ' + sing_image.as_posix() + \
    ' heudiconv' + \
    ' -d ' + df_row.dicom_template + \
    ' -s ' + df_row.bids_subj + \
    ' -ss ' + df_row.bids_ses + \
    ' -f ' + heuristics_script + \
    ' -b' 
    
    if output_dir is not None:
        output_dir = Path(output_dir).as_posix()
        cmd = cmd + ' -o ' + output_dir 

    if conv_dir is not None:
        conv_dir = Path(conv_dir).as_posix()
        cmd = cmd + ' --conv-outdir ' + conv_dir 
        
    if minmeta:
        cmd = cmd + ' --minmeta'

    if conversion:
        cmd = cmd + ' -c dcm2niix' 
    else:
        
        cmd = cmd + ' -c none' 
        
    return cmd

##  Run heudiconv without conversion

### Generate swarm commands

In [ ]:
df_sing = (
    df_bids.
    assign(
        cmd = lambda df:
        generate_singularity_command(df,
                                     project_dir = project_dir_absolute,
                                     output_dir=outdir_gen,
                                     heuristics_script= default_heuristics,
                                     conversion = False,
                                     sing_image = sing_image))
          )

swarm_path_gen = Path(scripts_dir).joinpath('heudiconv_generic_swarm' + analysis_version + '.cmd')

# not all commands resolve to a single dicom so getting unique ones before writing swarm
swarm_path_gen.write_text('\n'.join(df_sing.cmd.drop_duplicates())) 
print(swarm_path_gen.read_text())

### Run swarm

In [ ]:
log_dir_gen = scripts_dir.joinpath('swarm_output', analysis_version +'_generic')
log_dir_gen

In [ ]:
job_id_gen = !swarm -f {swarm_path_gen} -g 10 --logdir {log_dir_gen} --partition nimh,norm
job_id_gen = job_id_gen[0]
job_id_gen

### Information obtained 

The dataframe displayed below shows the information from the dicom headers. This information can be used by a custom heuristics file to convert dicoms to specific modalities in the BIDS structure.

In [ ]:
info_text_paths_gen = list(output_of_heudiconv_gen.glob('**/info/dicominfo.txt'))
info_text_paths_gen


In [ ]:
df_info_gen = pd.concat([pd.read_csv(p, sep = '\t').assign(file_path=p) for p in info_text_paths_gen]).reset_index(drop = True)
df_info_gen

In [ ]:
set(df_info_gen.series_description.values)


###  Checking the swarm output

Sometimes specific runs fail (observed on dashboard). File these in as 'files of interest':

In [ ]:
files_of_interest = []

In [ ]:
df_error_files_paths = pd.DataFrame([x.as_posix() for x in log_dir_gen.glob('*.e')],columns=['paths'])
df_error_files_paths = (df_error_files_paths.
                  loc[df_error_files_paths.paths.str.find(job_id_gen)>0,:].
                  assign(run = lambda df:
                         df.paths.str.extract(
                             '/.*swarm_\d*_(\d*).e',
                             expand=False).astype(int)).
                  sort_values('run'))
df_error_files_paths

if not files_of_interest:
    files_of_interest = list(range(len(df_error_files_paths)))




print('\n\n\n'.join(np.array(df_error_files_paths.paths)[files_of_interest]))

In [ ]:
error_files = [Path(x).read_text() for x in np.array(df_error_files_paths.paths)]

In [ ]:
print('\n\n\n'.join(error_files))

In [ ]:
output_files = [Path(x).with_suffix('.o').read_text() for x in np.array(df_error_files_paths.paths)]

In [ ]:
print('\n\n\n'.join(output_files))

In [ ]:
# df_dicoms.query('bids_subj in ["324","108"]')

In [ ]:
# %%bash
# module load afni
# dicom_hdr ???.dcm

## Heudiconv with custom heuristics script

###  Check heuristics

Heudiconv requires a heuristics file (created in the next section) in order to map the dicom files' metadata to the bids output structure. This is documented at the  nipy/heudiconv github repository. The file contains two main parts:
1. Templates create using the "create_key" function that specify where each run type belongs
2. The specification of the heuristic to categorise each run in the dicom tar.

The template is quite stereotyped and the examples on github are useful in figuring out how to write them.

The heuristic for categorising the runs is a little more challenging. Often the series description from the dicom header can be enough to categorise the scans:


In [ ]:
series_descriptions = set(df_info_gen.series_description)
series_descriptions

The heuristic is written as a boolean python expression that queries "s" a named tuple with a number of fields that are extracted from the dicom header. Here we will triple quote the heuristics so that we can pass the expression as a string but this is not required in our heuristics file:

In [ ]:
mprage_heuristic = """(' MP-Rage 1 mm' == s.series_description)"""

In [ ]:
def check_heuristic(s,heuristic):
    return eval(heuristic)

def check_heuristic_df(df,heuristic):
    matching = []
    for s in df.itertuples():
        result = check_heuristic(s,heuristic)
        if result:
            matching.append(s)
    return pd.DataFrame(matching)

In [ ]:
check_heuristic_df(df_info_gen, mprage_heuristic)

In [ ]:
memprage = """('ME-MP-RAGE 1mm PROMO' == s.series_description)"""
check_heuristic_df(df_info_gen, memprage)

In [ ]:
dti = """(s.series_description.startswith('edti'))"""
check_heuristic_df(df_info_gen, dti)


### fmrif heuristics file

In [ ]:
heuristics_script.write_text("""
# coding: utf-8
import os


def create_key(template, outtype=('dicom', 'nii.gz'), annotation_classes=None):
    if template is None or not template:
        raise ValueError('Template must be a valid format string')
    return template, outtype, annotation_classes

def infotodict(seqinfo):

    mprage = create_key('anat/sub-{subject}_{session}_acq-MPRAGE_run-{item:03d}_T1w')
    flair = create_key('anat/sub-{subject}_{session}_run-{item:03d}_FLAIR')
    MEMPRAGE1mmPROMO = create_key('anat/sub-{subject}_{session}_acq-MEMPRAGE1mmPROMO_run-{item:03d}_T1w')
    t1SE  = create_key('anat/sub-{subject}_{session}_acq-SE_run-{item:03d}_T1w')
    t2fatsat = create_key('anat/sub-{subject}_{session}_acq-fatsat_run-{item:03d}_T2w')
    FRFSE = create_key('anat/sub-{subject}_{session}_acq-FRFSE_run-{item:03d}_T2w')
    CUBE = create_key('anat/sub-{subject}_{session}_acq-CUBE_run-{item:03d}_T2w')
    rest = create_key('func/sub-{subject}_{session}_task-rest_run-{item:02d}_bold')
    dti = create_key('dwi/sub-{subject}_{session}_run-{item:02d}_dwi')
    ref = create_key('anat/sub-{subject}_{session}_acq-ref_fr8_run-{item:03d}_T1w')
    
    info = {mprage: [], MEMPRAGE1mmPROMO: [], t1SE: [],
    t2fatsat: [], flair: [], FRFSE: [], rest:[],
    CUBE:[],dti: [], ref: []}

   
    for s in seqinfo:
        if ('Ax T2 FLAIR' == s.series_description):
             info[flair].append(s.series_number)
        if ('Ax T2 FRFSE' == s.series_description):
             info[FRFSE].append(s.series_number)
        if ('FMRIF EPI 3mm iso RS' == s.series_description):
             info[rest].append(s.series_number)
        if ('ME-MP-RAGE 1mm PROMO' == s.series_description):
             info[MEMPRAGE1mmPROMO].append(s.series_number)
        if (' MP-Rage 1 mm' == s.series_description):
             info[mprage].append(s.series_number)
        if ('Sag CUBE T2' == s.series_description):
             info[CUBE].append(s.series_number)
        if ('Sag T1 Spin echo' == s.series_description):
             info[t1SE].append(s.series_number)
        if ('T2_1.7mm_fat_sat' == s.series_description):
             info[t2fatsat].append(s.series_number)
        if (s.series_description.startswith('edti')):
            info[dti].append(s.series_number)
        if ('Sagittal Ref PA fr8' == s.series_description):
             info[ref].append(s.series_number)
        if ('Sagittal Ref body fr8' == s.series_description):
            info[ref].append(s.series_number)
    return info
""")

###  Generate heudiconv swarm commands

In [ ]:
df_sing = (
    df_bids.
    assign(
        cmd = lambda df:
        generate_singularity_command(df,
                                     project_dir = project_dir_absolute,
                                     output_dir=outdir,
                                     heuristics_script= heuristics_script,
                                     conversion = True,
                                     sing_image = sing_image))
          )

swarm_path= Path(scripts_dir).joinpath('heudiconv_swarm' + analysis_version + '.cmd')

# not all commands resolve to a single dicom so getting unique ones before writing swarm
swarm_path.write_text('\n'.join(df_sing.cmd.drop_duplicates())) 
print(swarm_path.read_text())

In [ ]:
(len(df_sing.cmd),len(df_sing.cmd.drop_duplicates()))

### Run heudiconv conversion swarm 

In [ ]:
log_dir = scripts_dir.joinpath('swarm_output', analysis_version)

In [ ]:
job_id = !swarm -f {swarm_path} -g 10 --logdir {log_dir} --partition nimh,norm
job_id = job_id[0]
job_id

###  Issues with conversion.

#### Swarm failures

Some files failed (observed on dashboard). File these in as 'files of interest':

In [ ]:
files_of_interest = []

In [ ]:
df_error_files_paths = pd.DataFrame([x.as_posix() for x in log_dir.glob('*.e')],columns=['paths'])

df_error_files_paths = (df_error_files_paths.
                  loc[df_error_files_paths.paths.str.find(job_id)>0,:].
                  assign(run = lambda df:
                         df.paths.str.extract(
                             '/.*swarm_\d*_(\d*).e',
                             expand=False).astype(int)).
                  sort_values('run'))

df_error_files_paths

if not files_of_interest:
    files_of_interest = list(range(len(df_error_files_paths)))




print('\n\n\n'.join(np.array(df_error_files_paths.paths)[files_of_interest]))

In [ ]:
error_files = [Path(x).read_text() for x in np.array(df_error_files_paths.paths)]

In [ ]:
print('\n\n\n'.join(error_files))

In [ ]:
output_files = [Path(x).with_suffix('.o').read_text() for x in np.array(df_error_files_paths.paths)]

In [ ]:
print('\n\n\n'.join(output_files))

## Output bids directory

In [ ]:
!ls -Rl {output_of_heudiconv}

In [ ]:
for f in list(output_of_heudiconv.glob('**/sub-002*MEMPRAGE1mmPROMO_run-001_T1w*json')):
    !echo {f}
    !cat {f}

In [ ]:
# %cp -R {output_of_heudiconv} {output_of_heudiconv.with_name('BIDS')}